# Initialization

In [ ]:
import os
from pathlib import Path
import signal
import time

# PyBoolNet initialization
from pyboolnet.repository import get_all_names, get_bnet
from pyboolnet.file_exchange import bnet2primes
from pyboolnet.trap_spaces import compute_trap_spaces

# BioLQM init
import biolqm

# trappist init
import trappist

# timeout with signals
class TimeOutException(Exception):
   pass

def handler():
    raise TimeOutException()
    
signal.signal(signal.SIGALRM, handler)

In [ ]:
# compute trapspaces using PyBoolNet directly
def bench_pyboolnet(models, max=1000, timeout=120):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            primes = bnet2primes(model + ".bnet")
            end = time.perf_counter()
            print(f"implicants: {end - start}s")
            tspaces = compute_trap_spaces(primes, "min", max_output=max)
            end = time.perf_counter()
            print(f"trap spaces {end - start}s\n===")
        except TimeOutException as ex:
            print("DNF")
        signal.alarm(0)

In [ ]:
# compute trapspaces using trappist
def bench_trappist(models, max=1000, timeout=120):
    for model in models:
        print(model)
        signal.alarm(timeout)
        try:
            start = time.perf_counter()
            biolqm.save(biolqm.load(model + ".bnet"), model + ".pnml")
            end = time.perf_counter()
            print(f"conversion: {end - start}s")
            trappist.compute_trap_spaces(model + ".pnml", max_output=max)
            end = time.perf_counter()
            print(f"trap spaces {end - start}s\n===")
        except TimeOutException as ex:
            print("DNF")
        signal.alarm(0)

## PyBoolNet repository

Unfortunately all models are very easy

In [ ]:
# all models from the PyBoolNet repository https://github.com/hklarner/pyboolnet/tree/master/pyboolnet/repository
models = get_all_names()

# only the slowest ones
models = ['jaoude_thdiff', 'n12c5', 'remy_tumorigenesis', 'selvaggio_emt']
# but we have an issue with the PNML conversion of "remy_tumorigenesis"
models = ['jaoude_thdiff', 'n12c5', 'selvaggio_emt']

# save .bnet files locally
for model in models:
    net = get_bnet(model)
    with open(model + ".bnet", "wt") as f:
        print(net, file=f)

In [ ]:
bench_pyboolnet(models)

In [ ]:
bench_trappist(models)

## Local set of big models

Collected from various bibliographic sources (see article for details)

In [ ]:
# all models from the trappist's benchmark directory
models = [str(Path().absolute() / "selected raw models" / name[:-5]) for name in os.listdir("selected raw models") if name.endswith(".bnet")]

In [ ]:
bench_pyboolnet(models)

In [ ]:
bench_trappist(models)